# K₇ Robustness Matrix - A100 Optimized

**Objectif**: Compléter la grille de robustesse avec N=20000

**Runtime**: GPU A100 recommandé

---

In [ ]:
# Cell 1: Setup
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
import json
from datetime import datetime
from dataclasses import dataclass, asdict
from typing import List, Dict
import itertools
import time

# Try GPU acceleration
try:
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU_AVAILABLE = True
    print("✓ CuPy available - GPU acceleration enabled")
except ImportError:
    GPU_AVAILABLE = False
    print("✗ CuPy not available - using CPU")

print(f"NumPy version: {np.__version__}")

In [ ]:
# Cell 2: Constants
DET_G = 65/32  # G₂ metric determinant
H_STAR = 99    # b₂ + b₃ + 1
B2 = 21
B3 = 77

# Grid to complete
FULL_GRID = {
    "N": [1000, 2000, 5000, 10000, 20000],
    "k": [15, 25, 40, 60],
    "laplacian": ["unnormalized", "random_walk", "symmetric"],
}

# Just N=20000 for completion
N20K_GRID = {
    "N": [20000],
    "k": [15, 25, 40, 60],
    "laplacian": ["symmetric"],  # Only symmetric (most stable)
}

SEEDS = [42, 123, 456]

print(f"H* = {H_STAR}")
print(f"Target: λ₁×H* = 13 = dim(G₂) - 1")

In [ ]:
# Cell 3: TCS Construction (optimized)

def sample_S3(n: int, seed: int) -> np.ndarray:
    """Uniform sampling on S³."""
    rng = np.random.default_rng(seed)
    q = rng.standard_normal((n, 4))
    return q / np.linalg.norm(q, axis=1, keepdims=True)

def geodesic_S3_batched(Q: np.ndarray, batch_size: int = 5000) -> np.ndarray:
    """Geodesic distance matrix on S³ - batched for memory efficiency."""
    n = Q.shape[0]
    D = np.zeros((n, n), dtype=np.float32)  # Use float32 to save memory
    
    for i in range(0, n, batch_size):
        end_i = min(i + batch_size, n)
        for j in range(0, n, batch_size):
            end_j = min(j + batch_size, n)
            dot = np.clip(np.abs(Q[i:end_i] @ Q[j:end_j].T), 0, 1)
            D[i:end_i, j:end_j] = 2 * np.arccos(dot)
    
    return D

def tcs_distance_matrix(n: int, ratio: float, seed: int) -> np.ndarray:
    """TCS distance matrix for K₇ ≈ S¹ × S³ × S³."""
    rng = np.random.default_rng(seed)
    
    # S¹ component
    theta = rng.uniform(0, 2*np.pi, n).astype(np.float32)
    theta_diff = np.abs(theta[:, None] - theta[None, :])
    d_S1_sq = np.minimum(theta_diff, 2*np.pi - theta_diff)**2
    
    # Two S³ components
    q1 = sample_S3(n, seed + 1000).astype(np.float32)
    q2 = sample_S3(n, seed + 2000).astype(np.float32)
    
    # Batched geodesic for large N
    batch_size = 5000 if n > 10000 else n
    d1 = geodesic_S3_batched(q1, batch_size)
    d2 = geodesic_S3_batched(q2, batch_size)
    
    # TCS combination
    alpha = DET_G / (ratio**3)
    d_sq = alpha * d_S1_sq + d1**2 + (ratio**2) * d2**2
    
    return np.sqrt(np.maximum(d_sq, 0))

def ratio_formula(H_star: int) -> float:
    """Ratio = H*/84 (standard formula)."""
    return max(H_star / 84, 0.8)

print("TCS functions ready ✓")

In [ ]:
# Cell 4: Graph Laplacian (optimized)

def compute_lambda1(D: np.ndarray, k: int = 25, 
                    laplacian_type: str = "symmetric") -> float:
    """Compute λ₁ from distance matrix."""
    n = D.shape[0]
    k = min(k, n - 1)
    
    # Sigma = median of k-NN distances
    knn_dists = np.partition(D, k, axis=1)[:, :k]
    sigma = max(np.median(knn_dists), 1e-10)
    
    # Gaussian weights
    W = np.exp(-D**2 / (2 * sigma**2))
    np.fill_diagonal(W, 0)
    
    # Keep only k nearest neighbors
    for i in range(n):
        idx = np.argpartition(W[i], -k)[-k:]
        mask = np.ones(n, dtype=bool)
        mask[idx] = False
        W[i, mask] = 0
    
    W = (W + W.T) / 2
    d = np.maximum(W.sum(axis=1), 1e-10)
    
    # Build Laplacian
    if laplacian_type == "unnormalized":
        L = np.diag(d) - W
    elif laplacian_type == "random_walk":
        d_inv = 1.0 / d
        L = np.eye(n) - np.diag(d_inv) @ W
    elif laplacian_type == "symmetric":
        d_inv_sqrt = 1.0 / np.sqrt(d)
        L = np.eye(n) - np.diag(d_inv_sqrt) @ W @ np.diag(d_inv_sqrt)
    else:
        raise ValueError(f"Unknown: {laplacian_type}")
    
    # Sparse eigensolve
    L_sparse = sp.csr_matrix(L)
    eigs, _ = eigsh(L_sparse, k=5, which='SM', tol=1e-8)
    eigs = np.sort(np.real(eigs))
    
    # λ₁ = first non-zero eigenvalue
    for ev in eigs:
        if ev > 1e-8:
            return float(ev)
    
    return float(eigs[1]) if len(eigs) > 1 else 0.0

print("Laplacian functions ready ✓")

In [ ]:
# Cell 5: Quick test
print("Quick test with N=1000...")
t0 = time.time()

ratio = ratio_formula(H_STAR)
D = tcs_distance_matrix(1000, ratio, 42)
l1 = compute_lambda1(D, k=25, laplacian_type="symmetric")
product = l1 * H_STAR

print(f"  λ₁×H* = {product:.4f} (target: 13)")
print(f"  Time: {time.time() - t0:.2f}s")
print("\n✓ Ready for full run!")

In [ ]:
# Cell 6: Run N=20000 grid
@dataclass
class Result:
    N: int
    k: int
    laplacian_type: str
    seed: int
    ratio: float
    lambda1: float
    lambda1_x_Hstar: float
    deviation_from_13_pct: float
    deviation_from_14_pct: float
    closer_to: str
    time_seconds: float

def run_grid(grid: dict, seeds: list, verbose: bool = True):
    """Run the full grid."""
    results = []
    ratio = ratio_formula(H_STAR)
    
    configs = list(itertools.product(
        grid["N"], grid["k"], grid["laplacian"], seeds
    ))
    total = len(configs)
    
    for i, (N, k, lap_type, seed) in enumerate(configs):
        if verbose:
            print(f"[{i+1}/{total}] N={N}, k={k}, {lap_type}, seed={seed}...", end=" ")
        
        t0 = time.time()
        try:
            D = tcs_distance_matrix(N, ratio, seed)
            l1 = compute_lambda1(D, k, lap_type)
            elapsed = time.time() - t0
            
            product = l1 * H_STAR
            dev_13 = abs(product - 13) / 13 * 100
            dev_14 = abs(product - 14) / 14 * 100
            
            result = Result(
                N=N, k=k, laplacian_type=lap_type, seed=seed,
                ratio=ratio, lambda1=l1, lambda1_x_Hstar=product,
                deviation_from_13_pct=dev_13, deviation_from_14_pct=dev_14,
                closer_to="13" if dev_13 < dev_14 else "14",
                time_seconds=elapsed
            )
            results.append(asdict(result))
            
            if verbose:
                print(f"λ₁×H*={product:.2f} ({result.closer_to}) [{elapsed:.1f}s]")
                
        except Exception as e:
            print(f"ERROR: {e}")
            results.append({"N": N, "k": k, "error": str(e)})
    
    return results

print("Runner ready ✓")

In [ ]:
# Cell 7: RUN N=20000 (main computation)
print("="*60)
print("  RUNNING N=20000 GRID")
print("="*60)
print(f"Grid: {N20K_GRID}")
print(f"Seeds: {SEEDS}")
print(f"Total tests: {len(N20K_GRID['N']) * len(N20K_GRID['k']) * len(N20K_GRID['laplacian']) * len(SEEDS)}")
print()

t_start = time.time()
results_20k = run_grid(N20K_GRID, SEEDS)
t_total = time.time() - t_start

print()
print(f"Total time: {t_total/60:.1f} minutes")

In [ ]:
# Cell 8: Analysis
print("="*60)
print("  ANALYSIS")
print("="*60)

valid = [r for r in results_20k if "error" not in r]

if valid:
    products = [r["lambda1_x_Hstar"] for r in valid]
    
    print(f"\nN=20000 Results:")
    print(f"  Mean λ₁×H*: {np.mean(products):.4f}")
    print(f"  Std: {np.std(products):.4f}")
    print(f"  Range: [{np.min(products):.2f}, {np.max(products):.2f}]")
    
    # By k
    print(f"\nBy k:")
    for k in N20K_GRID["k"]:
        subset = [r for r in valid if r["k"] == k]
        if subset:
            prods = [r["lambda1_x_Hstar"] for r in subset]
            print(f"  k={k}: {np.mean(prods):.4f} ± {np.std(prods):.4f}")
    
    # Closest to 13
    closest = min(valid, key=lambda r: r["deviation_from_13_pct"])
    print(f"\nClosest to 13:")
    print(f"  k={closest['k']}: λ₁×H*={closest['lambda1_x_Hstar']:.4f} ({closest['deviation_from_13_pct']:.2f}% dev)")

In [ ]:
# Cell 9: Save results
output = {
    "metadata": {
        "timestamp": datetime.now().isoformat(),
        "H_star": H_STAR,
        "grid": N20K_GRID,
        "seeds": SEEDS,
        "total_time_minutes": t_total / 60,
    },
    "results": results_20k,
    "summary": {
        "count": len(valid),
        "mean_product": float(np.mean(products)) if valid else None,
        "std_product": float(np.std(products)) if valid else None,
    }
}

filename = "K7_N20000_results.json"
with open(filename, "w") as f:
    json.dump(output, f, indent=2)

print(f"\n✓ Saved: {filename}")
print("\nDownload this file and share with Claude!")

In [ ]:
# Cell 10: Display for copy-paste
print("="*60)
print("  QUICK SUMMARY (copy this)")
print("="*60)
print()
print(f"N=20000 Results:")
for r in valid:
    print(f"  k={r['k']}, seed={r['seed']}: λ₁×H* = {r['lambda1_x_Hstar']:.4f} ({r['closer_to']})")